In [1]:
## importing required libraries
import os
import shutil
import random

!pip install ultralytics
!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.7/611.7 kB 8.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_path_img = "./yolo_data_CSD/images/train/"
train_path_label = "./yolo_data_CSD/labels/train/"
val_path_img = "./yolo_data_CSD/images/val/"
val_path_label = "./yolo_data_CSD/labels/val/"
test_path = "./yolo_data_CSD/test"

In [5]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images

    for i in files:
      if '(1)' in i:
        files.remove(i)

    print(files)

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue

      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/Car_sign_detection_yolov8/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
['00068', '00022', '00144', '00125', '00045', '00119', '00091', '00069', '00064', '00032', '00146', '00111', '00029', '00065', '00054', '00036', '00089', '00079', '00097', '00055', '00067', '00120', '00016', '00007', '00080', '00034', '00003', '00010', '00075', '00084', '00015', '00102', '00042', '00109', '00138', '00130', '00122', '00107', '00027', '00021', '00106', '00024', '00128', '00103', '00000', '00004', '00012', '00076', '00112', '00025', '00124', '00053', '00011', '00148', '00134', '00099', '00082', '00008', '00062', '00092', '00117', '00060', '00095', '00033', '00044', '00113', '00114', '00048', '00009', '00115', '00041', '00039', '00071', '00047', '00147', '00150', '00070', '00019', '00030', '00057', '00038', '00142', '00058', '00006', '00140', '00133', '00093', '00017', '00100', '00073', '00141', '00040', '00002', '00126', '00051', '00135', '00129', '00063', '00105', '00104', '00149', '00077', '00123', '00035', '00059', '00090', '00118', '0004

  0%|          | 0/119 [00:00<?, ?it/s]

------ Training data created with 80% split 119 images -------


  0%|          | 0/29 [00:00<?, ?it/s]

------ Testing data created with a total of 29 images ----------
------ TASK COMPLETED -------


In [6]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.120 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.2/78.2 GB disk)


In [7]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/Car_sign_detection_yolov8/dataset.yaml epochs=20 imgsz=640 batch=8 project=/content/drive/MyDrive/Car_sign_detection_yolov8/training_results name=carSigns

100% 21.5M/21.5M [00:00<00:00, 33.3MB/s]
Ultralytics YOLOv8.0.120 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Car_sign_detection_yolov8/dataset.yaml, epochs=20, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Car_sign_detection_yolov8/training_results, name=carSigns, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=F

In [8]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/Car_sign_detection_yolov8/training_results/carSigns/weights/best.pt conf=0.77 source=/content/drive/MyDrive/Car_sign_detection_yolov8/test_images

Ultralytics YOLOv8.0.120 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127132 parameters, 0 gradients

image 1/28 /content/drive/MyDrive/Car_sign_detection_yolov8/test_images/road100.png: 640x640 (no detections), 16.4ms
image 2/28 /content/drive/MyDrive/Car_sign_detection_yolov8/test_images/road104.png: 608x640 (no detections), 99.0ms
image 3/28 /content/drive/MyDrive/Car_sign_detection_yolov8/test_images/road105.png: 640x480 (no detections), 104.6ms
image 4/28 /content/drive/MyDrive/Car_sign_detection_yolov8/test_images/road109.png: 448x640 1 prohibitory, 109.4ms
image 5/28 /content/drive/MyDrive/Car_sign_detection_yolov8/test_images/road110.png: 448x640 (no detections), 12.9ms
image 6/28 /content/drive/MyDrive/Car_sign_detection_yolov8/test_images/road115.png: 384x640 (no detections), 104.7ms
image 7/28 /content/drive/MyDrive/Car_sign_detection_yolov8/test_images/road121.png: 512x640 (no detections), 97.4ms
image 8/28 /content/dri

In [10]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/Car_sign_detection_yolov8/output

In [ ]:
#!yolo task=detect mode=predict model=/content/drive/MyDrive/Car_sign_detection_yolov8/training_results/carSigns/weights/best.pt conf=0.77 source=/content/drive/MyDrive/Car_sign_detection_yolov8/video

Ultralytics YOLOv8.0.120 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127132 parameters, 0 gradients

video 1/2 (1/2575) /content/drive/MyDrive/Car_sign_detection_yolov8/video/pexels-tom-fisk-4832678-2560x1440-30fps.mp4: 384x640 (no detections), 81.0ms
video 1/2 (2/2575) /content/drive/MyDrive/Car_sign_detection_yolov8/video/pexels-tom-fisk-4832678-2560x1440-30fps.mp4: 384x640 (no detections), 18.0ms
video 1/2 (3/2575) /content/drive/MyDrive/Car_sign_detection_yolov8/video/pexels-tom-fisk-4832678-2560x1440-30fps.mp4: 384x640 (no detections), 31.4ms
video 1/2 (4/2575) /content/drive/MyDrive/Car_sign_detection_yolov8/video/pexels-tom-fisk-4832678-2560x1440-30fps.mp4: 384x640 (no detections), 23.2ms
video 1/2 (5/2575) /content/drive/MyDrive/Car_sign_detection_yolov8/video/pexels-tom-fisk-4832678-2560x1440-30fps.mp4: 384x640 (no detections), 42.1ms
video 1/2 (6/2575) /content/drive/MyDrive/Car_sign_detection_yolov8/video/pexels-tom-fis

In [ ]:
#!cp -r /content/runs/detect/predict5 /content/drive/MyDrive/Car_sign_detection_yolov8/output